In [1]:
import cv2
import os

import paths
from process_dataset import common

In [2]:
# %pip install -U openmim

# !mim install "mmcv>=2.0.0rc1,<2.1.0"

# %cd $paths.mm_parent_dirpath
# # !git clone https://github.com/open-mmlab/mmdetection.git
# %cd mmdetection
# %pip install -e .
# %cd $paths.proj_path

# %cd $paths.mm_parent_dirpath
# # !git clone https://github.com/open-mmlab/mmengine
# %cd mmengine
# %pip install -e .
# %cd $paths.proj_path

In [3]:
# Epoch number (to choose a checkpoint)
e = 8

device = "cuda:0"
# device = "cpu"

score_thr = 0.5
# score_thr = 0

# Video input filename
input_filename = "day_hq.mp4"

output_name = input_filename.split(".")[0] + f"_e{e}_{score_thr}"

# Video output filename
output_filename = output_name + ".mp4"

# Step (every n-th frame will be taken, other ignored)
# step = 1
step = 5

input_fps = 30

# Frames output folder
frames_dirname = output_name

# Pick a specific .pth file
paths.last_checkpoint_filepath = os.path.join(paths.working_dirpath, f"epoch_{e}.pth")

# stop_at_frames = 0
stop_at_frames = 1000

In [4]:
print(paths.model_config_filepath)
print(paths.last_checkpoint_filepath)

c:/Users/z004ktej/Desktop/d_projekty/bp/proj\..\mmyolo\configs\yolov8\yolov8_s_syncbn_fast_8xb16-500e_coco_custom.py
c:/Users/z004ktej/Desktop/d_projekty/bp/proj\working_dir_yolov8\epoch_8.pth


Error `UserWarning: Class names are not saved in the checkpoint's meta data, use COCO classes by default.`
can be ignored. Just now, even after receiving this error, images are annotated correctly with my classes.

In [5]:
import mmcv
from mmdet.apis import init_detector, inference_detector

# MMYolo integration
from mmyolo.utils import register_all_modules
from mmyolo.registry import VISUALIZERS
register_all_modules()

model = init_detector(paths.model_config_filepath, paths.last_checkpoint_filepath, device=device)

visualizer = VISUALIZERS.build(model.cfg.visualizer)
visualizer.dataset_meta["classes"] = tuple(common.classes_ids.keys())

# I wanted to use the original yolox config and just adjust the number of 
# classes and classes themselves here, but this strangely results in misdetections
# when inferencing... So, in this script, it is necessary to use a custom config
# where the number of classes is updated
# model.bbox_head.num_classes = len(common.classes)

video = mmcv.VideoReader(input_filename)

frames_dir = os.path.join(paths.working_dirpath, frames_dirname)
if not os.path.exists(frames_dir):
    os.mkdir(frames_dir)

counter = 0
try:
    for frame in video:
        print(counter // step, end="\r")

        if counter % step == 0:

            out_filepath = os.path.join(frames_dir, str(counter // step).zfill(6) + ".jpg")

            frame = mmcv.imconvert(frame, "bgr", "rgb")

            result = inference_detector(model, frame)

            visualizer.add_datasample(
                os.path.basename(out_filepath),
                frame,
                data_sample=result,
                draw_gt=False,
                show=False,
                wait_time=0,
                out_file=out_filepath,
                pred_score_thr=score_thr
            )

            # Without mmyolo, this worked:
            # if score_thr == 0:
            #     model.show_result(frame, result, out_file=out_filepath)
            # else:
            #     model.show_result(frame, result, score_thr=score_thr, out_file=out_filepath)

        counter += 1
        if counter == stop_at_frames * step: # Doesn't stop if stop_at_frames == 0
            raise KeyboardInterrupt

except KeyboardInterrupt:
    print("Stop requested")

c:\Users\z004ktej\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


02/14 11:12:24 - mmengine - WARNING - The "model" registry in mmyolo did not set import location. Fallback to call `mmyolo.utils.register_all_modules` instead.
02/14 11:12:25 - mmengine - WARNING - The "model" registry in mmdet did not set import location. Fallback to call `mmdet.utils.register_all_modules` instead.
02/14 11:12:25 - mmengine - WARNING - The "task util" registry in mmyolo did not set import location. Fallback to call `mmyolo.utils.register_all_modules` instead.
02/14 11:12:25 - mmengine - WARNING - The "task util" registry in mmdet did not set import location. Fallback to call `mmdet.utils.register_all_modules` instead.
Loads checkpoint by local backend from path: c:/Users/z004ktej/Desktop/d_projekty/bp/proj\working_dir_yolov8\epoch_8.pth
02/14 11:12:25 - mmengine - WARNING - The "visualizer" registry in mmyolo did not set import location. Fallback to call `mmyolo.utils.register_all_modules` instead.
02/14 11:12:25 - mmengine - WARNING - The "visualizer" registry in mmd

c:\users\z004ktej\desktop\d_projekty\bp\mmengine\mmengine\visualization\visualizer.py:163: UserWarning: `Visualizer` backend is not initialized because save_dir is None.
  warnings.warn('`Visualizer` backend is not initialized '


02/14 11:12:25 - mmengine - WARNING - The "transform" registry in mmyolo did not set import location. Fallback to call `mmyolo.utils.register_all_modules` instead.
02/14 11:12:25 - mmengine - WARNING - The "transform" registry in mmdet did not set import location. Fallback to call `mmdet.utils.register_all_modules` instead.


c:\Users\z004ktej\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


c:\users\z004ktej\desktop\d_projekty\bp\mmengine\mmengine\visualization\visualizer.py:707: UserWarning: Warning: The bbox is out of bounds, the drawn bbox may not be in the image
  warnings.warn(
c:\users\z004ktej\desktop\d_projekty\bp\mmengine\mmengine\visualization\visualizer.py:778: UserWarning: Warning: The polygon is out of bounds, the drawn polygon may not be in the image
  warnings.warn(


Stop requested


In [6]:
fps = input_fps // step
mmcv.frames2video(frames_dir, os.path.join(paths.working_dirpath, output_filename), fps=fps)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1000/1000, 70.4 task/s, elapsed: 14s, ETA:     0s
